In [3]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader , DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import  PromptTemplate
from langchain.llms import CTransformers
import pypdf
import os
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone

c:\Users\Cheth\anaconda3\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
PINECONE_API_KEY="22bdf7d2-2365-4da4-b27c-221dc2220db2"
PINECONE_API_ENV="us-east-1"

In [9]:
#Extract the data from the pgf

def load_pdf(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents
    

In [10]:

extarcted_data=load_pdf("data/")

In [11]:
def text_split(extarcted_data):
    text_splliter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splliter.split_documents(extarcted_data)
    return text_chunks 

In [12]:
text_chunks=text_split(extarcted_data)

In [14]:
len(text_chunks)

7020

In [5]:
def down_embedding():
    embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embedding

embedding=down_embedding()

In [4]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [5]:
query_result=embedding.embed_query("Hello world")

In [6]:
len(query_result)

384

In [6]:
import os
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

from pinecone import Pinecone
pc = Pinecone()

In [16]:



pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("medical-chatbot")
docsearch=PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embedding=embedding, index_name="medical-chatbot")

In [7]:
index_name="medical-chatbot"
docsearch=PineconeVectorStore.from_existing_index(index_name, embedding)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of drugs commonly ca

In [18]:
print(docs[0].page_content)
print(docs[1].page_content)
print(docs[2].page_content)

GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies
Allergic rhinitis is commonly triggered by
exposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.
The presence of an allergen causes the
body's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.
IgE molecules attach to mast
cells, which contain histamine.HistaminePollen grains
Lymphocyte
FIRST EXPOSURE
allergens are the following:
• plant pollens
• animal fur and dander
• body parts from house mites (microscopic creatures
found in all houses)
• house dust• mold spores• cigarette smoke• solvents• cleaners
Common food allergens include the following:
• nuts, especially peanuts, walnuts, and brazil nuts
• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives
The following types of drugs commonly cause aller-
gic reactions:
• penicillin or other antibiotics
itchy, scratch

In [8]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [9]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [13]:
llm=CTransformers(model="../model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [14]:
docsearch = PineconeVectorStore.from_existing_index(index_name, embedding)


In [15]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

ValidationError: 1 validation error for RetrievalQA
retriever
  Can't instantiate abstract class BaseRetriever with abstract methods _aget_relevant_documents, _get_relevant_documents (type=type_error)

In [1]:


def generate_response(query):
    # Retrieve relevant documents and generate response
    result = qa({"query": query})
    
    # Extract the answer and source documents from the result
    answer = result["result"]
    source_documents = result["source_documents"]
    
    return answer, source_documents

# Example usage
query = "What are the symptoms of fertility ?"
answer, source_documents = generate_response(query)
print("Answer:", answer)
print("Source Documents:", source_documents)

NameError: name 'qa' is not defined

In [16]:
pip show langchain

Name: langchain
Version: 0.0.225
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: c:\users\cheth\anaconda3\envs\mchatbot\lib\site-packages
Requires: aiohttp, async-timeout, dataclasses-json, langchainplus-sdk, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community
Note: you may need to restart the kernel to use updated packages.


In [20]:
!pip install --upgrade langchain

  Using cached langchain-0.2.10-py3-none-any.whl.metadata (6.9 kB)
Using cached langchain-0.2.10-py3-none-any.whl (990 kB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.225
    Uninstalling langchain-0.0.225:
      Successfully uninstalled langchain-0.0.225


In [18]:
pip show langchain

Name: langchain
Version: 0.0.225
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: c:\users\cheth\anaconda3\envs\mchatbot\lib\site-packages
Requires: aiohttp, async-timeout, dataclasses-json, langchainplus-sdk, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community
Note: you may need to restart the kernel to use updated packages.
